In [1]:
import numpy as np

def birthday_experiment(class_size, n_sim=1000):
    """Simulates the birthday paradox. Vectorized = Fast!"""
    sims = np.random.randint(1, 365 + 1, (n_sim, class_size))
    sort_sims = np.sort(sims, axis=1)
    n_uniq = (sort_sims[:, 1:] != sort_sims[:, :-1]).sum(axis = 1) + 1
    return {"est_prob": np.mean(n_uniq != class_size)}

In [2]:
%%time
results = [birthday_experiment(class_size=size, n_sim=10_000) for size in range(2, 100)]

CPU times: user 1.07 s, sys: 143 ms, total: 1.21 s
Wall time: 1.1 s


In [3]:
import ray
ray.init(num_cpus=2);

2023-01-23 15:01:34,160	INFO worker.py:1538 -- Started a local Ray instance.


In [4]:
@ray.remote
def birthday_experiment(class_size, n_sim=1000):
    """Simulates the birthday paradox. Vectorized = Fast!"""
    sims = np.random.randint(1, 365 + 1, (n_sim, class_size))
    sort_sims = np.sort(sims, axis=1)
    n_uniq = (sort_sims[:, 1:] != sort_sims[:, :-1]).sum(axis = 1) + 1
    return {"est_prob": np.mean(n_uniq != class_size)}

futures = [birthday_experiment.remote(class_size=size, n_sim=10_000) for size in range(2, 10)]

In [5]:
%%time
results = ray.get(futures)

CPU times: user 564 µs, sys: 281 µs, total: 845 µs
Wall time: 748 µs


In [6]:
results

[{'est_prob': 0.003},
 {'est_prob': 0.0099},
 {'est_prob': 0.0162},
 {'est_prob': 0.0269},
 {'est_prob': 0.0416},
 {'est_prob': 0.0582},
 {'est_prob': 0.0754},
 {'est_prob': 0.0894}]

In [7]:
ray.shutdown()